In [29]:
import pandas as pd
import openai

In [44]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

### Read the Sampled Dataset with Amazon Inventory Data

In [4]:
df_items = pd.read_json('../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl', lines=True)

In [38]:
df_items.head(3)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,image
0,Camera & Photo,80x100 Monocular-Telescope Low Night Vision Mo...,3.6,187,[],80x100 Monocular-Telescope Low Night Vision Mo...,29.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': '80X100 monocular telescope high po...,MANDDDWU,"[Electronics, Camera & Photo, Binoculars & Sco...",{'Product Dimensions': '3.94 x 3.94 x 3.94 inc...,B09YHBXZC8,NaN,NaN,NaN,https://m.media-amazon.com/images/I/51Ni+NLJS4...
1,All Electronics,"SoundPEATS Air Conduction Headphones, RunFree ...",4.0,132,[𝗨𝗻𝗯𝗲𝗮𝘁𝗮𝗯𝗹𝗲 𝗖𝗼𝗺𝗳𝗼𝗿𝘁 𝗮𝗻𝗱 𝗟𝗶𝗴𝗵𝘁𝘄𝗲𝗶𝗴𝗵𝘁 𝗗𝗲𝘀𝗶𝗴𝗻 - 🍀...,"SoundPEATS Air Conduction Headphones, RunFree ...",29.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Soundpeats RunFree Lite - The Best...,SoundPEATS,"[Electronics, Headphones, Earbuds & Accessorie...",{'Package Dimensions': '7.48 x 5.16 x 2.13 inc...,B0BRXDR9GF,NaN,NaN,NaN,https://m.media-amazon.com/images/I/31OGeYLIfn...
2,Computers,"𝟮𝟬𝟮𝟯 𝗟𝐚𝐭𝐞𝐬𝐭 Tablet 10.1"" Octa-Core Android 11 ...",4.3,411,[【VALUE FOR MONEY】Get all the bundled accessor...,"𝟮𝟬𝟮𝟯 𝗟𝐚𝐭𝐞𝐬𝐭 Tablet 10.1"" Octa-Core Android 11 ...",179.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'The most complete tablet bundle', ...",FEONAL,"[Electronics, Computers & Accessories, Compute...","{'Standing screen display size': '10.1', 'Scre...",B0B4C98K37,NaN,NaN,NaN,https://m.media-amazon.com/images/I/51HfEQ1Osc...


In [6]:
list(df_items["features"].items())[1]

(1,
 ['𝗨𝗻𝗯𝗲𝗮𝘁𝗮𝗯𝗹𝗲 𝗖𝗼𝗺𝗳𝗼𝗿𝘁 𝗮𝗻𝗱 𝗟𝗶𝗴𝗵𝘁𝘄𝗲𝗶𝗴𝗵𝘁 𝗗𝗲𝘀𝗶𝗴𝗻 - 🍀 𝘽𝙚𝙨𝙩 𝙑𝙖𝙡𝙪𝙚 𝙊𝙥𝙚𝙣-𝙚𝙖𝙧 𝙎𝙥𝙤𝙧𝙩 𝙃𝙚𝙖𝙙𝙥𝙝𝙤𝙣𝙚 - 𝘾𝙉𝙀𝙏 🍀. The RunFree Lite headset is carefully optimized for weight distribution, ensuring a secure and comfortable fit that eliminates worries of slipping and shaking. Wrapped in skin-friendly liquid silicone, these lightweight headphones (0.99oz) provide a weightless and comfortable experience for extended listening during gym sessions, exercise, running, and more.',
  '𝗢𝗽𝗲𝗻-𝗘𝗮𝗿 𝗗𝗲𝘀𝗶𝗴𝗻 𝗮𝗻𝗱 𝗗𝘆𝗻𝗮𝗺𝗶𝗰 𝗦𝗼𝘂𝗻𝗱 - Stay connected and aware of your surroundings with our open-ear design sports headphones, which use air conduction technology to enhance the aerodynamic transmission of sound, resulting in a more dynamic and powerful audio experience. Enjoy up to 17 hours of playtime on a single charge to keep your music going for daily and weekly exercises. Only 1-2 hours charge will get full energy back.',
  '𝗣𝗼𝘄𝗲𝗿𝗳𝘂𝗹 𝗕𝗮𝘀𝘀 𝗮𝗻𝗱 𝗦𝘁𝗮𝗯𝗹𝗲 𝗖𝗼𝗻𝗻𝗲𝗰𝘁𝗶𝗼𝗻 - Featuring the latest low-frequency enhancement technology and B

### Preprocess Title and Features

In [7]:
def preprocess_description(row):
    return f"{row['title']} {' '.join(row['features'])}"

In [10]:
def extract_first_last_image(row):
    return row["images"][0].get("large", "")

In [12]:
df_items["description"] = df_items.apply(preprocess_description, axis=1)
df_items["image"] = df_items.apply(extract_first_last_image, axis=1)

In [13]:
df_items.head(3)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,image
0,Camera & Photo,80x100 Monocular-Telescope Low Night Vision Mo...,3.6,187,[],80x100 Monocular-Telescope Low Night Vision Mo...,29.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': '80X100 monocular telescope high po...,MANDDDWU,"[Electronics, Camera & Photo, Binoculars & Sco...",{'Product Dimensions': '3.94 x 3.94 x 3.94 inc...,B09YHBXZC8,NaN,NaN,NaN,https://m.media-amazon.com/images/I/51Ni+NLJS4...
1,All Electronics,"SoundPEATS Air Conduction Headphones, RunFree ...",4.0,132,[𝗨𝗻𝗯𝗲𝗮𝘁𝗮𝗯𝗹𝗲 𝗖𝗼𝗺𝗳𝗼𝗿𝘁 𝗮𝗻𝗱 𝗟𝗶𝗴𝗵𝘁𝘄𝗲𝗶𝗴𝗵𝘁 𝗗𝗲𝘀𝗶𝗴𝗻 - 🍀...,"SoundPEATS Air Conduction Headphones, RunFree ...",29.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Soundpeats RunFree Lite - The Best...,SoundPEATS,"[Electronics, Headphones, Earbuds & Accessorie...",{'Package Dimensions': '7.48 x 5.16 x 2.13 inc...,B0BRXDR9GF,NaN,NaN,NaN,https://m.media-amazon.com/images/I/31OGeYLIfn...
2,Computers,"𝟮𝟬𝟮𝟯 𝗟𝐚𝐭𝐞𝐬𝐭 Tablet 10.1"" Octa-Core Android 11 ...",4.3,411,[【VALUE FOR MONEY】Get all the bundled accessor...,"𝟮𝟬𝟮𝟯 𝗟𝐚𝐭𝐞𝐬𝐭 Tablet 10.1"" Octa-Core Android 11 ...",179.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'The most complete tablet bundle', ...",FEONAL,"[Electronics, Computers & Accessories, Compute...","{'Standing screen display size': '10.1', 'Scre...",B0B4C98K37,NaN,NaN,NaN,https://m.media-amazon.com/images/I/51HfEQ1Osc...


In [21]:
list(df_items["description"].items())[0]

(0,
 '80x100 Monocular-Telescope Low Night Vision Monoculars High Definition for Adults High Powered with Smartphone Adapter Monocular Telescope Hunting Wildlife Bird Watching Travel Camping Hiking ')

In [22]:
list(df_items["image"].items())[0]

(0, 'https://m.media-amazon.com/images/I/51Ni+NLJS4L._AC_.jpg')

### Sample 50 items from the dataset

In [23]:
df_sample = df_items.sample(50, random_state=42)

In [24]:
len(df_sample)

50

In [25]:
data_to_embed = df_sample[["description", "image", "rating_number", "price", "average_rating", "parent_asin"]].to_dict(orient="records")

In [28]:
data_to_embed[0]

{'description': "USB C Hub, MCY USB C to HDMI Multiptort Adapter, 10 in 1 Portable Type C Dongle with 4K HDMI, VGA, PD Charging, USB 3.0 and USB 2.0 Ports, SD/TF Card, Compatible with MacBook, HP, Dell and More 【10 IN 1 USB C Docking Station】MCY Type-C smart docking station can solve 10 kinds of office troubles at one time. USB C hub multiport adapter hub rich interface. HDMI makes the big screen clearer. 1G files are transferred in 3 seconds, and there is no need to wait for the transfer. 【Wonderful more than one side】10-port USB C to HDMI adapter turns your laptop into a thin and light desktop computer. It is equipped with 4K@60Hz HDMI port, VGA port, SD/TF card reader and USB 3.0 and USB 2.0 ports. Get rid of the problem of insufficient notebook ports and connect more devices. A Type C interface helps you upgrade your space. 【4K Video HDMI to USB C Hub】Mirror or extend your laptop/tablet/phone's screen with hdmi port and directly stream 4K@60Hz (3840 x 2160) or full HD 1080P lifelik

### Define the Emebeddings Function

In [30]:
response = openai.embeddings.create(input="random text it is", model="text-embedding-3-small")

In [36]:
print(f"Length of the embedding: {len(response.data[0].embedding)}")
print(response.data[0].embedding)

Length of the embedding: 1536
[0.00036091305082663894, 0.017657894641160965, 0.008908487856388092, 0.00041161986882798374, -0.013839972205460072, -0.07973096519708633, -0.012917307205498219, -0.006283665541559458, -0.003098085755482316, -0.005289414431899786, 0.02301889657974243, 0.004911599215120077, -0.03286595642566681, 0.0031617176719009876, 0.03401133418083191, 0.0086141899228096, -0.012949123047292233, -0.027918564155697823, -0.052814606577157974, 0.02413245663046837, 0.023098435252904892, 0.024784686043858528, -0.018007872626185417, 5.319241972756572e-05, -0.0017200539587065578, 0.015597807243466377, -0.00448605976998806, 0.013617259450256824, 0.04603779315948486, -0.03964276984333992, 0.0016226173611357808, -0.049633003771305084, 0.027298152446746826, -0.028602609410881996, 0.007671639788895845, -0.006243895273655653, 0.015955738723278046, 0.012050320394337177, 0.023543860763311386, -0.0016951977740973234, -0.022557564079761505, -0.05007842928171158, 0.03059111163020134, 0.0326

In [37]:
def get_embeddings(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(input=text, model="text-embedding-3-small")
    return response.data[0].embedding

### Qdrant Client and Collection Creation

In [40]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [43]:
qdrant_client.create_collection(
    collection_name="amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

### Embed Data in Qdrant Collection

#### Test PointStruct

In [45]:
pointstruct = PointStruct(
    id=0,
    vector=get_embeddings("Test Description"),
    payload={'description': "Test Description",
    'model': "text-embedding-3-small"},
)

In [46]:
pointstruct

PointStruct(id=0, vector=[-0.02241450548171997, -0.009639437310397625, 0.004973499570041895, -0.03153634071350098, -0.01113973930478096, -0.005382332019507885, 0.0037582553923130035, -0.016848387196660042, 0.03723748400807381, 0.040208082646131516, 0.018228664994239807, -0.036967430263757706, -0.029901010915637016, -0.014260366559028625, 0.004350874572992325, 0.029630957171320915, 0.028415711596608162, 0.011709854006767273, -0.018513722345232964, 0.06811369210481644, 0.021544329822063446, -0.0012386864982545376, 0.005239803344011307, -0.025025030598044395, -0.01801862195134163, -0.060312122106552124, -0.009256861172616482, 0.0039045349694788456, 0.036247286945581436, -0.05965198948979378, 0.06925392150878906, -0.04008805751800537, 0.0037132464349269867, -0.0009358131210319698, 0.02316465601325035, 0.02217445708811283, 0.036427322775125504, 0.037177473306655884, 0.002471746876835823, -0.010832177475094795, -0.004887232556939125, -0.02685539796948433, -0.005303565878421068, -0.0255801416

#### Amazon Data

In [47]:
pointstructs = []
for i, data in enumerate(data_to_embed):
    embedding = get_embeddings(data["description"])
    pointstructs.append(PointStruct(
        id=i,
        vector=embedding,
        payload=data,
    ))

In [48]:
pointstructs

[PointStruct(id=0, vector=[-0.034248899668455124, -0.02096666768193245, -0.027667975053191185, 0.0493970662355423, -0.032122138887643814, -0.008426794782280922, -0.009916531853377819, 0.03519190102815628, 0.02810937911272049, -0.02568165957927704, -0.01197808701545, 0.03282437101006508, -0.044019971042871475, 0.01759595051407814, -0.008030534721910954, -0.0033606858924031258, -0.07684434205293655, 0.005166427697986364, 0.025220192968845367, 0.023253940045833588, 0.0031675719656050205, 0.04606647789478302, 0.050962042063474655, 0.010694003663957119, 0.014265359379351139, 0.010874577797949314, -0.04426073655486107, 0.016793398186564445, -0.014175072312355042, 0.03200175613164902, 0.006641116458922625, -0.02293291874229908, 0.01676330156624317, -0.0013731160433962941, -0.03747917339205742, 0.032683923840522766, 0.009349728934466839, 0.005176459439098835, 0.01977287232875824, 0.009871387854218483, 0.015108038671314716, 0.012078405357897282, 0.0777672752737999, -0.031901437789201736, -0.000

### Write Embedded Data to Qdrant

In [49]:
qdrant_client.upsert(
    collection_name="amazon-items-collection-00",
    points=pointstructs,
    wait=True,
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

### Query Qdrant: Define a Function for Data Retrieval

In [50]:
def retrieve_data(query, k=5):
    query_embedding = get_embeddings(query)
    results = qdrant_client.query_points(
        collection_name="amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )
    return results

#### Test Retrieval

In [53]:
query_response=retrieve_data("What kind of Earphones can I get", k=10)

In [55]:
query_response.points

[ScoredPoint(id=20, version=0, score=0.5061893, payload={'description': 'Wireless Earbud, Bluetooth 5.1 Headphones with Microphone Deep Bass Bluetooth Earphones in-Ear, CVC8.0 Noise Cancelling Earbud for Sport Running Gym IPX7 Waterproof, 30H Playtime, Touch Control White ', 'image': 'https://m.media-amazon.com/images/I/41AmQP5zJ0L._AC_.jpg', 'rating_number': 457, 'price': None, 'average_rating': 4.1, 'parent_asin': 'B09F36P17Y'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=23, version=0, score=0.48579925, payload={'description': 'USB C Earphone for Google Pixel 7 Pro 7A6 6A, Type C Headphones in Ear Wired Earbuds Magnetic Noise Canceling Headset Microphone for iPad 10 Samsung Galaxy S23 S22 Ultra S21 FE S20 A54 Oneplus Green USB C Headphone: Widely compatible with Samsung Galaxy S23 Ultra 2023/S23 Plus S23+/S23/S22 Ultra/S22 Plus S22+/S22/S21 FE/S21/S21+/S21 Ultra/S20 FE/S20/S20 Ultra/S20 Plus/Galaxy Z Fold 4 3/Galaxy Z Flip 4 3/A53/A54/A33/A34, for iPad 10,iPad Ai